#Importing the necessary packages 

In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

#Opening CSV files and converting them to dataframes for further operations

In [8]:
%cd "C:\Users\deeparam\Desktop\DMCA\Sols_DSP_Capstone_Projects_PS_Chegg_2019\2. WALMART STORE PREDICTION - FORECASTING"

C:\Users\deeparam\Desktop\DMCA\Sols_DSP_Capstone_Projects_PS_Chegg_2019\2. WALMART STORE PREDICTION - FORECASTING


In [9]:
train = pd.read_csv('train.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')

#Merging 'train', 'features' and 'stores' dataset -- further splitting the merged file for 'training' and 'validation' datasets

In [10]:
train_m1 = train.merge(features, on=['Store','Date','IsHoliday'],how='left').fillna(0)
train_merge = train_m1.merge(stores, on=['Store'], how='left').fillna(0)

#The last 450 rows of data are split off into a validation dataset for our further use during validations
train_data = train_merge.head(len(train_merge) - 450)

train_validate = train_merge.tail(450)

In [11]:
train_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421570 entries, 0 to 421569
Data columns (total 16 columns):
Store           421570 non-null int64
Dept            421570 non-null int64
Date            421570 non-null object
Weekly_Sales    421570 non-null float64
IsHoliday       421570 non-null bool
Temperature     421570 non-null float64
Fuel_Price      421570 non-null float64
MarkDown1       421570 non-null float64
MarkDown2       421570 non-null float64
MarkDown3       421570 non-null float64
MarkDown4       421570 non-null float64
MarkDown5       421570 non-null float64
CPI             421570 non-null float64
Unemployment    421570 non-null float64
Type            421570 non-null object
Size            421570 non-null int64
dtypes: bool(1), float64(10), int64(3), object(2)
memory usage: 51.9+ MB


In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421120 entries, 0 to 421119
Data columns (total 16 columns):
Store           421120 non-null int64
Dept            421120 non-null int64
Date            421120 non-null object
Weekly_Sales    421120 non-null float64
IsHoliday       421120 non-null bool
Temperature     421120 non-null float64
Fuel_Price      421120 non-null float64
MarkDown1       421120 non-null float64
MarkDown2       421120 non-null float64
MarkDown3       421120 non-null float64
MarkDown4       421120 non-null float64
MarkDown5       421120 non-null float64
CPI             421120 non-null float64
Unemployment    421120 non-null float64
Type            421120 non-null object
Size            421120 non-null int64
dtypes: bool(1), float64(10), int64(3), object(2)
memory usage: 51.8+ MB


In [13]:
train_validate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 421120 to 421569
Data columns (total 16 columns):
Store           450 non-null int64
Dept            450 non-null int64
Date            450 non-null object
Weekly_Sales    450 non-null float64
IsHoliday       450 non-null bool
Temperature     450 non-null float64
Fuel_Price      450 non-null float64
MarkDown1       450 non-null float64
MarkDown2       450 non-null float64
MarkDown3       450 non-null float64
MarkDown4       450 non-null float64
MarkDown5       450 non-null float64
CPI             450 non-null float64
Unemployment    450 non-null float64
Type            450 non-null object
Size            450 non-null int64
dtypes: bool(1), float64(10), int64(3), object(2)
memory usage: 56.7+ KB


In [14]:
train_validate.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
421120,45,94,2012-04-27,3109.40,False,53.20,4.004,2872.86,0.03,37.12,74.97,2495.29,190.868006,8.567,B,118221
421121,45,94,2012-05-04,3123.83,False,55.21,3.951,11984.62,0.00,47.52,6150.63,1775.54,190.922212,8.567,B,118221
421122,45,94,2012-05-11,3800.78,False,61.24,3.889,12611.18,0.00,21.36,1667.39,2313.12,190.976417,8.567,B,118221
421123,45,94,2012-05-18,5918.72,False,66.30,3.848,6813.74,0.00,13.86,936.49,2941.55,190.996448,8.567,B,118221
421124,45,94,2012-05-25,2878.50,False,67.21,3.798,5370.39,0.00,361.22,1287.62,2461.81,191.002810,8.567,B,118221


#Cleaning merged data and preparing it for training models

In [15]:
#Since this is weekly sales - the date field is converted into Week of Year
train_data['Date'] = pd.to_datetime(train_data['Date'])
train_data['Year'] = train_data['Date'].dt.year
train_data['Week'] = train_data['Date'].dt.week
#function reference from stackoverflow
train_data['YearWeek'] = train_data.Year.astype(str).str.cat(train_data.Week.astype(str))


C:\Users\deeparam\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\deeparam\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\deeparam\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [16]:
#Also data is standardized as int data with #get dummies function so that we can use it in models

train_data.drop(['Date', 'Year', 'Week'], axis=1, inplace=True)
train_data['YearWeek'] = train_data.YearWeek.astype(int)
train_data = pd.get_dummies(train_data)
train_data

C:\Users\deeparam\AppData\Local\Continuum\anaconda2\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\deeparam\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,YearWeek,Type_A,Type_B,Type_C
0,1,1,24924.50,False,42.31,2.572,0.00,0.00,0.00,0.00,0.00,211.096358,8.106,151315,20105,1,0,0
1,1,1,46039.49,True,38.51,2.548,0.00,0.00,0.00,0.00,0.00,211.242170,8.106,151315,20106,1,0,0
2,1,1,41595.55,False,39.93,2.514,0.00,0.00,0.00,0.00,0.00,211.289143,8.106,151315,20107,1,0,0
3,1,1,19403.54,False,46.63,2.561,0.00,0.00,0.00,0.00,0.00,211.319643,8.106,151315,20108,1,0,0
4,1,1,21827.90,False,46.50,2.625,0.00,0.00,0.00,0.00,0.00,211.350143,8.106,151315,20109,1,0,0
5,1,1,21043.39,False,57.79,2.667,0.00,0.00,0.00,0.00,0.00,211.380643,8.106,151315,201010,1,0,0
6,1,1,22136.64,False,54.58,2.720,0.00,0.00,0.00,0.00,0.00,211.215635,8.106,151315,201011,1,0,0
7,1,1,26229.21,False,51.45,2.732,0.00,0.00,0.00,0.00,0.00,211.018042,8.106,151315,201012,1,0,0
8,1,1,57258.43,False,62.27,2.719,0.00,0.00,0.00,0.00,0.00,210.820450,7.808,151315,201013,1,0,0
9,1,1,42960.91,False,65.86,2.770,0.00,0.00,0.00,0.00,0.00,210.622857,7.808,151315,201014,1,0,0


In [17]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421120 entries, 0 to 421119
Data columns (total 18 columns):
Store           421120 non-null int64
Dept            421120 non-null int64
Weekly_Sales    421120 non-null float64
IsHoliday       421120 non-null bool
Temperature     421120 non-null float64
Fuel_Price      421120 non-null float64
MarkDown1       421120 non-null float64
MarkDown2       421120 non-null float64
MarkDown3       421120 non-null float64
MarkDown4       421120 non-null float64
MarkDown5       421120 non-null float64
CPI             421120 non-null float64
Unemployment    421120 non-null float64
Size            421120 non-null int64
YearWeek        421120 non-null int32
Type_A          421120 non-null uint8
Type_B          421120 non-null uint8
Type_C          421120 non-null uint8
dtypes: bool(1), float64(10), int32(1), int64(3), uint8(3)
memory usage: 48.2 MB


#The target variable is identified along with the predictor variables.  Data is split so as to train and validate the models.  The split is 82% - train, 18% - test.

In [18]:
y = targets = labels = train_data["Weekly_Sales"].values

columns = ["Store", "Dept", "YearWeek", "IsHoliday", "CPI", "Unemployment", "Size", "Temperature", "Fuel_Price", "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5", "Type_A", "Type_B", "Type_C"]
features = train_data[list(columns)].values
features

array([[1L, 1L, 20105, ..., 1, 0, 0],
       [1L, 1L, 20106, ..., 1, 0, 0],
       [1L, 1L, 20107, ..., 1, 0, 0],
       ...,
       [45L, 94L, 201214, ..., 0, 1, 0],
       [45L, 94L, 201215, ..., 0, 1, 0],
       [45L, 94L, 201216, ..., 0, 1, 0]], dtype=object)

In [19]:
features.dtype

dtype('O')

In [20]:
#Shuffle and split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.18, random_state=1)

# Exploring 4 different models and determining the best model for the prediction

##Instantiate and Fit the Decision Tree Regressor model

In [21]:
regressor = DecisionTreeRegressor(max_depth=32, random_state=0)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=32, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

#Decision Tree Regressor model accuracy Score

In [22]:
regressor.score(X_test, y_test)

0.9210390524962895

#Listing the hyper-parameters/feature importances

In [23]:
list(zip(columns, regressor.feature_importances_))

[('Store', 0.05296829000572885),
 ('Dept', 0.6293425505391305),
 ('YearWeek', 0.02506755229170134),
 ('IsHoliday', 0.0030326989083894417),
 ('CPI', 0.02790962232003494),
 ('Unemployment', 0.011484705181047815),
 ('Size', 0.18396767818469342),
 ('Temperature', 0.015878605289541537),
 ('Fuel_Price', 0.011383741010612913),
 ('MarkDown1', 0.0012210274713202543),
 ('MarkDown2', 0.0016972173272374676),
 ('MarkDown3', 0.01709172822799941),
 ('MarkDown4', 0.0025549196810581865),
 ('MarkDown5', 0.001845283786334389),
 ('Type_A', 0.0026018861719991904),
 ('Type_B', 0.011713703962702776),
 ('Type_C', 0.00023878964046758112)]

#Instantiate and Fit the Gradient Boosting Regressor

In [24]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

#Gradient Boosting Regressor accuracy Score

In [25]:
gbr.score(X_test, y_test)

0.7414631959871238

#Listing the hyper-parameters/feature importances

In [26]:
list(zip(columns, gbr.feature_importances_))

[('Store', 0.02430086535829197),
 ('Dept', 0.7463098321770615),
 ('YearWeek', 0.0007896636080571996),
 ('IsHoliday', 0.0008618417757848924),
 ('CPI', 0.009197936639667456),
 ('Unemployment', 0.0003930648435351271),
 ('Size', 0.19012369620074598),
 ('Temperature', 0.0005666315306227941),
 ('Fuel_Price', 0.0),
 ('MarkDown1', 0.0),
 ('MarkDown2', 0.0),
 ('MarkDown3', 0.009970382062287102),
 ('MarkDown4', 0.00014003044303921268),
 ('MarkDown5', 0.0),
 ('Type_A', 0.00011674800158876386),
 ('Type_B', 0.0158066567358815),
 ('Type_C', 0.0014226506234364781)]

#Instantiate and Fit the Random Forest Regressor

In [27]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

C:\Users\deeparam\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

#Random Forest Regressor accuracy Score

In [28]:
rfr.score(X_test, y_test)

0.9582642987926325

#Listing the hyper-parameters/feature importances

In [29]:
list(zip(columns, rfr.feature_importances_))

[('Store', 0.0558241029229977),
 ('Dept', 0.6212824481812086),
 ('YearWeek', 0.02213938464807826),
 ('IsHoliday', 0.0031962931634644137),
 ('CPI', 0.02942052683539793),
 ('Unemployment', 0.011917374719411044),
 ('Size', 0.18910286761275008),
 ('Temperature', 0.017487594481982303),
 ('Fuel_Price', 0.010936891088829708),
 ('MarkDown1', 0.0016676493470225387),
 ('MarkDown2', 0.001656468928689964),
 ('MarkDown3', 0.017662686120793983),
 ('MarkDown4', 0.0019037629515017925),
 ('MarkDown5', 0.0018943494150100101),
 ('Type_A', 0.0027354250159895743),
 ('Type_B', 0.01073451196354377),
 ('Type_C', 0.00043766260332831435)]

#Instantiate and Fit the Linear Regression Model

In [30]:
linreg = LinearRegression()
linreg.fit(X_train, y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

#Linear Regression Model accuracy Score and listing the co-efficients/hyper-parameters

In [31]:
linreg.score(X_test, y_test)

0.09171247456317766

In [32]:
list(zip(columns, linreg.coef_))

[('Store', -142.38789697460865),
 ('Dept', 109.65019310930111),
 ('YearWeek', 0.004404201997338577),
 ('IsHoliday', 481.7584897796239),
 ('CPI', -24.902788707166895),
 ('Unemployment', -187.26807883486205),
 ('Size', 0.0966760936707935),
 ('Temperature', 4.4330291718254635),
 ('Fuel_Price', -499.7880278196677),
 ('MarkDown1', 0.0023123143830326123),
 ('MarkDown2', 0.0051442053564218675),
 ('MarkDown3', 0.11537808930904352),
 ('MarkDown4', 0.03933468355398962),
 ('MarkDown5', 0.09348035346117564),
 ('Type_A', -1756.6540691258188),
 ('Type_B', -2197.274399753803),
 ('Type_C', 3953.928468879627)]


#Based on the above 4 models - the Linear Regression Model had the lowest Co-efficient of Determination : at around 9% model accuracy. The Decision Tree Regressor Model had about 92% model accuracy (Co-efficient of Determination).  The Random Forest Regressor Model showed 96% model accuracy.  

#The Gradient Boosting Regressor gave a model accuracy of 74% for a default max_depth parameter value of 3.  Besides this model takes the most time to train.  The training time increases, along with the increase in the value of the max_depth parameter.

#Based on all of this the Random Forest Regressor is found to have the best accuracy with optimal time to build the model


#Prepare the validation dataset data.  Tweak the Date field as we did in earlier step for training data, add missing columns expected by the models and initialize them

In [33]:
train_validate['Date'] = pd.to_datetime(train_validate['Date'])
train_validate['Year'] = train_validate['Date'].dt.year
train_validate['Week'] = train_validate['Date'].dt.week
train_validate['YearWeek'] = train_validate.Year.astype(str).str.cat(train_validate.Week.astype(str))
train_validate.drop(['Date', 'Year', 'Week'], axis=1, inplace=True)
train_validate['YearWeek'] = train_validate.YearWeek.astype(int)
train_validate = pd.get_dummies(train_validate)
train_validate['Type_A'] = 0
train_validate['Type_C'] = 0
train_validate

C:\Users\deeparam\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\deeparam\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\deeparam\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,YearWeek,Type_B,Type_A,Type_C
421120,45,94,3109.40,False,53.20,4.004,2872.86,0.03,37.12,74.97,2495.29,190.868006,8.567,118221,201217,1,0,0
421121,45,94,3123.83,False,55.21,3.951,11984.62,0.00,47.52,6150.63,1775.54,190.922212,8.567,118221,201218,1,0,0
421122,45,94,3800.78,False,61.24,3.889,12611.18,0.00,21.36,1667.39,2313.12,190.976417,8.567,118221,201219,1,0,0
421123,45,94,5918.72,False,66.30,3.848,6813.74,0.00,13.86,936.49,2941.55,190.996448,8.567,118221,201220,1,0,0
421124,45,94,2878.50,False,67.21,3.798,5370.39,0.00,361.22,1287.62,2461.81,191.002810,8.567,118221,201221,1,0,0
421125,45,94,3521.59,False,74.48,3.742,10643.62,48.20,201.56,2599.35,1856.97,191.009171,8.567,118221,201222,1,0,0
421126,45,94,3226.48,False,64.30,3.689,8429.61,148.60,76.15,1465.54,7180.97,191.015533,8.567,118221,201223,1,0,0
421127,45,94,3327.28,False,71.93,3.620,8148.19,252.70,90.05,1226.11,2942.39,191.029973,8.567,118221,201224,1,0,0
421128,45,94,2633.76,False,74.22,3.564,5565.31,222.50,1.16,2149.76,7474.76,191.064610,8.567,118221,201225,1,0,0
421129,45,94,3688.13,False,75.22,3.506,3291.36,425.60,0.00,314.88,2255.34,191.099246,8.567,118221,201226,1,0,0


#Mark out columns from the validate dataset that would be fed to the predictor model

In [34]:
columns2 = ["Store", "Dept", "YearWeek", "IsHoliday", "CPI", "Unemployment", "Size", "Temperature", "Fuel_Price", "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5", "Type_A", "Type_B", "Type_C"]
features_validate = train_validate[list(columns2)].values
features_validate

array([[45L, 94L, 201217, ..., 0L, 1, 0L],
       [45L, 94L, 201218, ..., 0L, 1, 0L],
       [45L, 94L, 201219, ..., 0L, 1, 0L],
       ...,
       [45L, 98L, 201241, ..., 0L, 1, 0L],
       [45L, 98L, 201242, ..., 0L, 1, 0L],
       [45L, 98L, 201243, ..., 0L, 1, 0L]], dtype=object)

#Execute the model on the validate dataset and see the predictions

In [35]:
pred_vals = rfr.predict(features_validate)

#Suffix the predicted column to the validation dataset so that we can do a comparison and assess the efficacy of the model

In [36]:
train_validate['Predicted'] = pred_vals
train_validate

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,YearWeek,Type_B,Type_A,Type_C,Predicted
421120,45,94,3109.40,False,53.20,4.004,2872.86,0.03,37.12,74.97,2495.29,190.868006,8.567,118221,201217,1,0,0,2944.413
421121,45,94,3123.83,False,55.21,3.951,11984.62,0.00,47.52,6150.63,1775.54,190.922212,8.567,118221,201218,1,0,0,2329.290
421122,45,94,3800.78,False,61.24,3.889,12611.18,0.00,21.36,1667.39,2313.12,190.976417,8.567,118221,201219,1,0,0,2902.729
421123,45,94,5918.72,False,66.30,3.848,6813.74,0.00,13.86,936.49,2941.55,190.996448,8.567,118221,201220,1,0,0,2826.588
421124,45,94,2878.50,False,67.21,3.798,5370.39,0.00,361.22,1287.62,2461.81,191.002810,8.567,118221,201221,1,0,0,2787.816
421125,45,94,3521.59,False,74.48,3.742,10643.62,48.20,201.56,2599.35,1856.97,191.009171,8.567,118221,201222,1,0,0,2929.515
421126,45,94,3226.48,False,64.30,3.689,8429.61,148.60,76.15,1465.54,7180.97,191.015533,8.567,118221,201223,1,0,0,2832.834
421127,45,94,3327.28,False,71.93,3.620,8148.19,252.70,90.05,1226.11,2942.39,191.029973,8.567,118221,201224,1,0,0,2832.834
421128,45,94,2633.76,False,74.22,3.564,5565.31,222.50,1.16,2149.76,7474.76,191.064610,8.567,118221,201225,1,0,0,3346.474
421129,45,94,3688.13,False,75.22,3.506,3291.36,425.60,0.00,314.88,2255.34,191.099246,8.567,118221,201226,1,0,0,3620.413


In [37]:
train_validate.drop(['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size', 'YearWeek', 'Type_B', 'Type_A', 'Type_C'], axis=1, inplace=True)
train_validate

,Store,Dept,Weekly_Sales,IsHoliday,Predicted
421120,45,94,3109.40,False,2944.413
421121,45,94,3123.83,False,2329.290
421122,45,94,3800.78,False,2902.729
421123,45,94,5918.72,False,2826.588
421124,45,94,2878.50,False,2787.816
421125,45,94,3521.59,False,2929.515
421126,45,94,3226.48,False,2832.834
421127,45,94,3327.28,False,2832.834
421128,45,94,2633.76,False,3346.474
421129,45,94,3688.13,False,3620.413


#Prepare the test dataset to run it by the model.  Merge the test data with the features and stores data

In [38]:
test = pd.read_csv('test.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')
test_m1 = test.merge(features, on=['Store','Date','IsHoliday'],how='left').fillna(0)
test_merge = test_m1.merge(stores, on=['Store'], how='left').fillna(0)

#The date variable is manipulated for further assessment.  The entire dataset is normalized

In [39]:
test_merge['Date'] = pd.to_datetime(test_merge['Date'])
test_merge['Year'] = test_merge['Date'].dt.year
test_merge['Week'] = test_merge['Date'].dt.week
test_merge['YearWeek'] = test_merge.Year.astype(str).str.cat(test_merge.Week.astype(str))
test_merge.drop(['Date', 'Year', 'Week'], axis=1, inplace=True)
test_merge['YearWeek'] = test_merge.YearWeek.astype(int)
test_merge = pd.get_dummies(test_merge)
test_merge

,Store,Dept,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,YearWeek,Type_A,Type_B,Type_C
0,1,1,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,151315,201244,1,0,0
1,1,1,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,151315,201245,1,0,0
2,1,1,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,151315,201246,1,0,0
3,1,1,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,151315,201247,1,0,0
4,1,1,False,52.34,3.207,2460.03,0.00,3838.35,150.57,6966.34,223.610984,6.573,151315,201248,1,0,0
5,1,1,False,64.12,3.198,6343.16,0.00,270.00,2928.90,10147.90,223.660021,6.573,151315,201249,1,0,0
6,1,1,False,48.89,3.168,3504.83,0.00,73.26,1636.80,2779.60,223.719277,6.573,151315,201250,1,0,0
7,1,1,False,56.02,3.098,8231.71,0.00,274.00,358.15,2834.02,223.839845,6.573,151315,201251,1,0,0
8,1,1,True,44.79,3.108,12659.55,37101.13,174.78,74.46,1208.86,223.960414,6.573,151315,201252,1,0,0
9,1,1,False,41.73,3.161,1214.08,25366.33,15.01,72.36,3940.02,224.080983,6.525,151315,20131,1,0,0


#Columns from the test dataset that would be used by the trained model to make a prediction of the target labels

In [40]:
columns3 =  ["Store", "Dept", "YearWeek", "IsHoliday", "Size", "Unemployment", "CPI", "Temperature", "Fuel_Price", "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5", "Type_A", "Type_B", "Type_C"]
features_test = test_merge[list(columns2)].values
features_test

array([[1L, 1L, 201244, ..., 1, 0, 0],
       [1L, 1L, 201245, ..., 1, 0, 0],
       [1L, 1L, 201246, ..., 1, 0, 0],
       ...,
       [45L, 98L, 201328, ..., 0, 1, 0],
       [45L, 98L, 201329, ..., 0, 1, 0],
       [45L, 98L, 201330, ..., 0, 1, 0]], dtype=object)

#the actual execution of the trained model on the test dataset.  Merge the predictions as a new column to the test dataset

In [41]:
test_vals = rfr.predict(features_test)
test_vals

array([30012.305, 21396.598, 29393.457, ...,   521.013,   519.365,
         468.061])

In [42]:
test_merge['Predicted_Weekly_Sales'] = test_vals
test_merge

,Store,Dept,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,YearWeek,Type_A,Type_B,Type_C,Predicted_Weekly_Sales
0,1,1,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,151315,201244,1,0,0,30012.305
1,1,1,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,151315,201245,1,0,0,21396.598
2,1,1,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,151315,201246,1,0,0,29393.457
3,1,1,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,151315,201247,1,0,0,45674.451
4,1,1,False,52.34,3.207,2460.03,0.00,3838.35,150.57,6966.34,223.610984,6.573,151315,201248,1,0,0,46754.327
5,1,1,False,64.12,3.198,6343.16,0.00,270.00,2928.90,10147.90,223.660021,6.573,151315,201249,1,0,0,24303.506
6,1,1,False,48.89,3.168,3504.83,0.00,73.26,1636.80,2779.60,223.719277,6.573,151315,201250,1,0,0,29061.155
7,1,1,False,56.02,3.098,8231.71,0.00,274.00,358.15,2834.02,223.839845,6.573,151315,201251,1,0,0,32104.123
8,1,1,True,44.79,3.108,12659.55,37101.13,174.78,74.46,1208.86,223.960414,6.573,151315,201252,1,0,0,29400.733
9,1,1,False,41.73,3.161,1214.08,25366.33,15.01,72.36,3940.02,224.080983,6.525,151315,20131,1,0,0,38810.616


In [43]:
list(zip(columns, rfr.feature_importances_))

[('Store', 0.0558241029229977),
 ('Dept', 0.6212824481812086),
 ('YearWeek', 0.02213938464807826),
 ('IsHoliday', 0.0031962931634644137),
 ('CPI', 0.02942052683539793),
 ('Unemployment', 0.011917374719411044),
 ('Size', 0.18910286761275008),
 ('Temperature', 0.017487594481982303),
 ('Fuel_Price', 0.010936891088829708),
 ('MarkDown1', 0.0016676493470225387),
 ('MarkDown2', 0.001656468928689964),
 ('MarkDown3', 0.017662686120793983),
 ('MarkDown4', 0.0019037629515017925),
 ('MarkDown5', 0.0018943494150100101),
 ('Type_A', 0.0027354250159895743),
 ('Type_B', 0.01073451196354377),
 ('Type_C', 0.00043766260332831435)]

# Conclusion and insights

#The Department and Size of the Store matter a lot to the Weekly Sales.  This is followed by Date, CPI, Unemployment, Temperature and Fuel Price.  Type B stores seem to have more sales as compared to Type A or Type C stores.  Also MarkDown 3 has the most impact on the sales as compared to other markdowns

#Other features don't seem to have a significant impact on the weekly sales